# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
from pytorch_tabular.utils import load_covertype_dataset
from rich.pretty import pprint
from plotly.subplots import make_subplots
from pytorch_tabular import TabularModel
import plotly.express as px
import torch
import random
import plotly.graph_objects as go
from scipy import stats
import shap
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.impute import KNNImputer
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import itertools
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from pytorch_tabular import model_sweep
from src.pt.model_sweep import model_sweep_custom
import warnings
from src.utils.configs import read_parse_config
from src.pt.hyper_opt import train_hyper_opt
from src.utils.hash import dict_hash
import pathlib
from tqdm import tqdm
import distinctipy
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from statannotations.Annotator import Annotator
from scipy.stats import mannwhitneyu
from plottable import ColumnDefinition, Table
from scipy.stats import chi2_contingency
from plottable.plots import bar
from plottable.cmap import normed_cmap, centered_cmap
import optuna
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
import matplotlib.cm
import matplotlib as mpl
from statsmodels.stats.multitest import multipletests
import re
import datetime
from collections import Counter
from matplotlib.ticker import MaxNLocator
from itertools import chain
from sklearn.metrics import mean_absolute_error
from scipy.stats import mannwhitneyu, variation, levene, zscore
import pyaging as pya
import matplotlib.lines as mlines
import statsmodels.formula.api as smf
from itertools import chain
from functools import reduce
import upsetplot
from src.plot.plotly_layout import add_layout
from pathlib import Path
import re
from openai import OpenAI
from matplotlib_venn import venn3, venn3_circles
import functools
from adjustText import adjust_text


def conjunction(conditions):
    return functools.reduce(np.logical_and, conditions)


def disjunction(conditions):
    return functools.reduce(np.logical_or, conditions)

# Figure 1

## Histograms

In [ ]:
path = 'E:/YandexDisk/Work/bbd/fmba/dnam/processed/special_63'
path_figures = 'E:/YandexDisk/Work/pydnameth/draft/13_fmba_cvd_dnam/figures'

df_our = pd.read_excel(f"{path}/funnorm/pheno_funnorm.xlsx", index_col=0)
colors_our = {
    'Control': 'chartreuse',
    'Case': 'red'
}

df_gse = pd.read_excel(f"{path}/GSE220622/gen/pheno_funnorm.xlsx", index_col=0)
colors_gse = {
    'Control': 'forestgreen',
    'Case': 'darkred'
}

datasets = {}
datasets['GSENEW'] = {
    'data': df_our,
    'colors': colors_our
}
datasets['GSE220622'] = {
    'data': df_gse,
    'colors': colors_gse
}

for ds_name, ds_dict in datasets.items():
    ds_data = ds_dict['data']
    hue_counts = ds_data['Status'].value_counts()
    hue_colors = ds_dict['colors']
    hue_replace = {x: f"{x} ({y})" for x, y in hue_counts.items()}
    hue_colors = {f"{x} ({y})": hue_colors[x] for x, y in hue_counts.items()}
    hue_order = [hue_replace[x] for x in ['Case', 'Control']]
    ds_data['Status'].replace(hue_replace, inplace=True)

    hist_bins = np.linspace(5, 115, 23)

    sns.set_theme(style='ticks')
    fig, ax = plt.subplots(figsize=(6, 3.5))
    histplot = sns.histplot(
        data=ds_data,
        bins=hist_bins,
        edgecolor='k',
        linewidth=1,
        x="Age",
        hue='Status',
        hue_order=hue_order,
        palette=hue_colors,
        ax=ax
    )
    histplot.set(xlim=(15, 70))
    # histplot.set_title(ds_name)
    plt.savefig(f"{path_figures}/1/age_hist_{ds_name}.png", bbox_inches='tight', dpi=200)
    plt.savefig(f"{path_figures}/1/age_hist_{ds_name}.pdf", bbox_inches='tight')
    plt.close(fig)

## Table for GSENEW

In [ ]:
path = 'E:/YandexDisk/Work/bbd/fmba/dnam/processed/special_63'
path_figures = 'E:/YandexDisk/Work/pydnameth/draft/13_fmba_cvd_dnam/figures'

df_our = pd.read_excel(f"{path}/funnorm/pheno_funnorm.xlsx", index_col=0)

special_diseases = [
    "Терапевт, I10",        # гипертония
    "Терапевт, I10.0",      # гипертония
    "Терапевт, I11",        # гипертония
    "Терапевт, I11.0",      # гипертония
    "Терапевт, I11.9",      # гипертония
    "Терапевт, I20",        # ИБС
    "Терапевт, I25",        # ИБС
    "Терапевт, I25.0",      # ИБС
    "Терапевт, E66",        # ожирение
    "Терапевт, E66.0",      # ожирение
    "Терапевт, E78",        # холестерин (гиперлипидемия, гиперхолестеринемия)
]

df_our['Target Column'] = np.where(disjunction([df_our[m] == 'Yes' for m in special_diseases]), 1, 0)

count_special = {
    'I10': df_our[(df_our['Терапевт, I10'] == 'Yes') | (df_our['Терапевт, I10.0'] == 'Yes')].shape[0],
    'I11': df_our[(df_our['Терапевт, I11'] == 'Yes') | (df_our['Терапевт, I11.0'] == 'Yes') | (df_our['Терапевт, I11.9'] == 'Yes')].shape[0],
    'I20': df_our[df_our['Терапевт, I20'] == 'Yes'].shape[0],
    'I25': df_our[(df_our['Терапевт, I25'] == 'Yes') | (df_our['Терапевт, I25.0'] == 'Yes')].shape[0],
    'E66': df_our[(df_our['Терапевт, E66'] == 'Yes') | (df_our['Терапевт, E66.0'] == 'Yes')].shape[0],
    'E78': df_our[df_our['Терапевт, E78'] == 'Yes'].shape[0],
}
print(count_special)

## Chronology of clocks

In [ ]:
path = 'E:/YandexDisk/Work/bbd/fmba/dnam/processed/special_63'
path_figures = 'E:/YandexDisk/Work/pydnameth/draft/13_fmba_cvd_dnam/figures'

df_clocks_info = pd.read_excel("E:/YandexDisk/Work/pydnameth/datasets/pyaging/clocks_meta_upd.xlsx", index_col='Clock Name')
df_clocks = pd.read_excel(f"{path}/funnorm/ages_corrected.xlsx", index_col=0)
df_clocks_info = df_clocks_info.loc[df_clocks.index.intersection(df_clocks_info.index), :].sort_values(["Chrolology"], ascending=[True])
df_clocks_info.to_excel(f"{path_figures}/1/ages.xlsx")

# Figure 2

## Aging clocks

In [ ]:
path = 'E:/YandexDisk/Work/bbd/fmba/dnam/processed/special_63/'
path_figures = 'E:/YandexDisk/Work/pydnameth/draft/13_fmba_cvd_dnam/figures'

df_pheno = pd.read_excel(f"{path}/funnorm/ages_data_corrected.xlsx", index_col=0)
df_ages = pd.read_excel(f"{path}/funnorm/ages_corrected.xlsx", index_col=0)
df_ages['Features'] = df_ages.index
df_ages['Mann-Whitney'] = -np.log10(df_ages['mannwhitneyu_pval_fdr_bh'].values)
df_ages['ANCOVA'] = -np.log10(df_ages['ancova_Special_Status[T.Control]_pval_fdr_bh'].values)

colors_tests = {
    'Mann-Whitney': 'fuchsia',
    'ANCOVA': 'cyan'
}

df_fig = df_ages.copy()
df_fig = df_fig.melt(id_vars='Features', value_vars=['Mann-Whitney', 'ANCOVA'], var_name='Test', value_name=r"$-\log_{10}(\mathrm{p-value})$")
sns.set_theme(style='ticks')
fig, ax = plt.subplots(figsize=(3, df_fig.shape[0] * 0.15))
barplot = sns.barplot(
    data=df_fig,
    y='Features',
    x=r"$-\log_{10}(\mathrm{p-value})$",
    edgecolor='black',
    palette=colors_tests,
    hue='Test',
    hue_order=['Mann-Whitney', 'ANCOVA'],
    ax=ax,
)
# ax.xaxis.tick_top()
# ax.xaxis.set_label_position('top')
ax.set_ylabel('')
ax.axvline(-np.log10(0.05), color="red", linestyle="dotted", linewidth=2.0)
plt.savefig(f"{path_figures}/2/ages_barplot.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path_figures}/2/ages_barplot.pdf", bbox_inches='tight')
plt.close(fig)

ages_target = df_ages.loc[(df_ages['mannwhitneyu_pval_fdr_bh'] < 0.05) | (df_ages['mannwhitneyu_pval_fdr_bh'] < 0.05), 'Features']

In [ ]:
colors_our = {
    'Control': 'chartreuse',
    'Case': 'red'
}

sns.set_theme(style='ticks')
fig = plt.figure(
    figsize=(11, 10),
    layout="constrained"
)
ncols = 2
nrows = 3
subfigs = fig.subfigures(
    ncols=ncols,
    nrows=nrows,
    wspace=0.1,
    hspace=0.1,
)
for epiage_id, epiage in enumerate(ages_target.index.values):
    row_id, col_id = divmod(epiage_id, 2)

    axs = subfigs[row_id, col_id].subplot_mosaic(
        [
            ['21', '22'],
        ],
        # height_ratios=[1, 4],
        width_ratios=[3, 1.5],
        gridspec_kw={
            #"bottom": 0.14,
            #"top": 0.95,
            # "left": 0.1,
            # "right": 0.5,
            #"wspace": 0.33,
            #"hspace": 0.01,
        },
    )
    
    xy_min = df_pheno[['Age', epiage]].min().min()
    xy_max = df_pheno[['Age', epiage]].max().max()
    xy_ptp = xy_max - xy_min
    bisect = sns.lineplot(
        x=[xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp],
        y=[xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp],
        linestyle='--',
        color='black',
        linewidth=1.0,
        ax=axs['21']
    )
    regplot = sns.regplot(
        data=df_pheno.loc[df_pheno['Status'] == 'Control', :],
        x='Age',
        y=epiage,
        color=colors_our['Control'],
        scatter=False,
        truncate=False,
        ax=axs['21']
    )
    scatter = sns.scatterplot(
        data=df_pheno,
        x='Age',
        y=epiage,
        hue='Status',
        palette=colors_our,
        linewidth=0.5,
        alpha=0.75,
        edgecolor="k",
        s=20,
        hue_order=list(colors_our.keys()),
        legend=True,
        ax=axs['21'],
    )
    axs['21'].set_xlim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
    axs['21'].set_ylim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
    
    sns.violinplot(
        data=df_pheno,
        x='Status',
        y=f"{epiage} acceleration",
        hue='Status',
        palette=colors_our,
        density_norm='width',
        order=['Control', 'Case'],
        saturation=0.75,
        linewidth=1.0,
        ax=axs['22'],
        legend=False,
        cut=0,
    )
    axs['22'].set_ylabel(f"{epiage} acceleration")

fig.savefig(f"{path_figures}/2/ages_distribution.png", bbox_inches='tight', dpi=200)
fig.savefig(f"{path_figures}/2/ages_distribution.pdf", bbox_inches='tight')

plt.close(fig)

## Aging metrics

In [ ]:
path = 'E:/YandexDisk/Work/bbd/fmba/dnam/processed/special_63/'
path_figures = 'E:/YandexDisk/Work/pydnameth/draft/13_fmba_cvd_dnam/figures'

df_pheno = pd.read_excel(f"{path}/funnorm/pheno_funnorm.xlsx", index_col=0)
df_pheno.rename(
    columns={
        'stemtoc': 'stemTOC',
        'epitoc1': 'epiTOC1',
        'dnamtl': 'DNAmTL',
        'dunedinpace': 'DunedinPACE',
        'pcdnamtl': 'PCDNAmTL',
        'zhangmortality': 'ZhangMortality',
    },
    inplace=True
)
df_metrics = pd.read_excel(f"{path}/funnorm/metrics.xlsx", index_col=0)
df_metrics['Features'] = df_metrics.index
df_metrics['Mann-Whitney'] = -np.log10(df_metrics['mannwhitneyu_pval_fdr_bh'].values)
df_metrics['ANCOVA'] = -np.log10(df_metrics['ancova_Special_Status[T.Control]_pval_fdr_bh'].values)

colors_tests = {
    'Mann-Whitney': 'fuchsia',
    'ANCOVA': 'cyan'
}

df_fig = df_metrics.copy()
df_fig = df_fig.melt(id_vars='Features', value_vars=['Mann-Whitney', 'ANCOVA'], var_name='Test', value_name=r"$-\log_{10}(\mathrm{p-value})$")
sns.set_theme(style='ticks')
fig, ax = plt.subplots(figsize=(3, df_fig.shape[0] * 0.15))
barplot = sns.barplot(
    data=df_fig,
    y='Features',
    x=r"$-\log_{10}(\mathrm{p-value})$",
    edgecolor='black',
    palette=colors_tests,
    hue='Test',
    hue_order=['Mann-Whitney', 'ANCOVA'],
    ax=ax,
)
# ax.xaxis.tick_top()
# ax.xaxis.set_label_position('top')
ax.set_ylabel('')
ax.axvline(-np.log10(0.05), color="red", linestyle="dotted", linewidth=2.0)
plt.savefig(f"{path_figures}/2/metrics_barplot.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path_figures}/2/metrics_barplot.pdf", bbox_inches='tight')
plt.close(fig)

metrics_target = df_metrics.loc[(df_metrics['mannwhitneyu_pval_fdr_bh'] < 0.05) | (df_metrics['mannwhitneyu_pval_fdr_bh'] < 0.05), 'Features']

In [ ]:
colors_our = {
    'Control': 'chartreuse',
    'Case': 'red'
}
    
n_rows = 2
n_cols = 2
fig_width = 5
fig_height = 8

sns.set_theme(style='ticks')
fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), gridspec_kw={'wspace':0.15, 'hspace': 0.15}, layout='constrained')
for epi_metric_id, epi_metric in enumerate(metrics_target):
    row_id, col_id = divmod(epi_metric_id, n_cols)
    
    ql = df_pheno[epi_metric].quantile(0.01)
    qh = df_pheno[epi_metric].quantile(0.99)
    
    sns.violinplot(
        # data=df_pheno.loc[(df_pheno[epi_metric] > ql) & (df_pheno[epi_metric] < qh), :],
        data=df_pheno,
        x='Status',
        y=epi_metric,
        palette=colors_our,
        scale='width',
        order=['Control', 'Case'],
        saturation=0.75,
        ax=axs[row_id, col_id],
        legend=False,
        cut=0,
    )
    axs[row_id, col_id].set_ylabel(epi_metric)
    axs[row_id, col_id].ticklabel_format(style='scientific', scilimits=(-1, 1), axis='y', useOffset=True, useMathText=True)

fig.savefig(f"{path_figures}/2/metrics_distribution.png", bbox_inches='tight', dpi=200)
fig.savefig(f"{path_figures}/2/metrics_distribution.pdf", bbox_inches='tight')
plt.close(fig)

## EpiScores

In [ ]:
path = 'E:/YandexDisk/Work/bbd/fmba/dnam/processed/special_63/'
path_figures = 'E:/YandexDisk/Work/pydnameth/draft/13_fmba_cvd_dnam/figures'

df_scores = pd.read_excel(f"{path}/funnorm/scores.xlsx", index_col=0)
df_scores.index = df_scores.index.astype(str)
df_scores.index = df_scores.index.str.replace(' (EpiScores)', '')
df_pheno = pd.read_csv(f"{path}/funnorm/episcores_Les_63_funnorm.csv", index_col=0)
df_pheno.index = df_pheno.index.str.replace('X', '')
df_pheno_add = pd.read_excel(f"{path}/funnorm/pheno_funnorm.xlsx", index_col=0)
df_pheno_add.index = df_pheno_add.index.astype(str)
df_pheno.loc[df_pheno.index, 'Status'] = df_pheno_add.loc[df_pheno.index, 'Status']

df_scores['Features'] = df_scores.index
df_scores[r"$-\log_{10}(\mathrm{Mann-Whitney})$"] = -np.log10(df_scores['mannwhitneyu_pval_fdr_bh'].values)
df_scores[r"$-\log_{10}(\mathrm{ANCOVA})$"] = -np.log10(df_scores['ancova_Status[T.Control]_pval_fdr_bh'].values)

episcores_nonsign = df_scores.index[(df_scores['mannwhitneyu_pval_fdr_bh'] > 0.05) & (df_scores['ancova_Status[T.Control]_pval_fdr_bh'] > 0.05)]
episcores_sign = df_scores.index.difference(episcores_nonsign)

sns.set_theme(style='ticks')
fig, ax = plt.subplots(figsize=(7, 6), gridspec_kw={}, layout='constrained')
scatter = sns.scatterplot(
    data=df_scores.loc[episcores_nonsign, :],
    x=r"$-\log_{10}(\mathrm{Mann-Whitney})$",
    y=r"$-\log_{10}(\mathrm{ANCOVA})$",
    linewidth=0.25,
    alpha=0.75,
    edgecolor="k",
    s=15,
    color='gray',
    ax=ax,
)
ax.axvline(-np.log10(0.05), color='lightgray', linestyle="-", linewidth=2.5, zorder=0)
ax.axvline(-np.log10(0.05), color='fuchsia', linestyle="dotted", linewidth=2.0, zorder=1)
ax.axhline(-np.log10(0.05), color='lightgray', linestyle="-", linewidth=2.5, zorder=0)
ax.axhline(-np.log10(0.05), color='cyan', linestyle="dotted", linewidth=2.0, zorder=1)
scatter = sns.scatterplot(
    data=df_scores.loc[episcores_sign, :],
    x=r"$-\log_{10}(\mathrm{Mann-Whitney})$",
    y=r"$-\log_{10}(\mathrm{ANCOVA})$",
    linewidth=0.25,
    alpha=0.75,
    edgecolor="k",
    s=25,
    color='red',
    ax=ax,
)
texts = []
for i, row in df_scores.loc[episcores_sign, :].iterrows():
    texts.append(ax.text(row[r"$-\log_{10}(\mathrm{Mann-Whitney})$"], row[r"$-\log_{10}(\mathrm{ANCOVA})$"], row['Features'], fontsize=9))
adjust_text(texts, arrowprops=dict(arrowstyle='-', color='black'))
fig.savefig(f"{path_figures}/2/episcores.png", bbox_inches='tight', dpi=200)
fig.savefig(f"{path_figures}/2/episcores.pdf", bbox_inches='tight')
plt.close(fig)

# Figure 4

In [ ]:
def check_for_nonnumeric(pd_series=None):
    if pd.to_numeric(pd_series, errors='coerce').isna().sum() == 0:
        return 0
    else:
        return 1


def gene_plot(d, geneid, lfc, lfc_thr, pv_thr, genenames, gfont, pv, gstyle):
    if genenames is not None and genenames == "deg":
        for i in d[geneid].unique():
            if (d.loc[d[geneid] == i, lfc].iloc[0] >= lfc_thr[0] and d.loc[d[geneid] == i, pv].iloc[0] < pv_thr[0]) or \
                    (d.loc[d[geneid] == i, lfc].iloc[0] <= -lfc_thr[1] and d.loc[d[geneid] == i, pv].iloc[0] < pv_thr[1]):
                if gstyle == 1:
                    plt.text(d.loc[d[geneid] == i, lfc].iloc[0], d.loc[d[geneid] == i, 'logpv_add_axy'].iloc[0], i,
                                  fontsize=gfont)
                elif gstyle == 2:
                    plt.annotate(i, xy=(d.loc[d[geneid] == i, lfc].iloc[0], d.loc[d[geneid] == i, 'logpv_add_axy'].iloc[0]),
                                 xycoords='data', xytext=(5, -15), textcoords='offset points', size=6,
                                 bbox=dict(boxstyle="round", alpha=0.1),
                                 arrowprops=dict(arrowstyle="wedge,tail_width=0.5", alpha=0.1, relpos=(0, 0)))
                else:
                    print("Error: invalid gstyle choice")
                    sys.exit(1)
    elif genenames is not None and type(genenames) is tuple:
        for i in d[geneid].unique():
            if i in genenames:
                if gstyle == 1:
                    plt.text(d.loc[d[geneid] == i, lfc].iloc[0], d.loc[d[geneid] == i, 'logpv_add_axy'].iloc[0], i,
                                  fontsize=gfont)
                elif gstyle == 2:
                    plt.annotate(i, xy=(d.loc[d[geneid] == i, lfc].iloc[0], d.loc[d[geneid] == i, 'logpv_add_axy'].iloc[0]),
                                 xycoords='data', xytext=(5, -15), textcoords='offset points', size=6,
                                 bbox=dict(boxstyle="round", alpha=0.1),
                                 arrowprops=dict(arrowstyle="wedge,tail_width=0.5", alpha=0.1, relpos=(0, 0)))
                else:
                    print("Error: invalid gstyle choice")
                    sys.exit(1)
    elif genenames is not None and type(genenames) is dict:
        for i in d[geneid].unique():
            if i in genenames:
                if gstyle == 1:
                    plt.text(d.loc[d[geneid] == i, lfc].iloc[0], d.loc[d[geneid] == i, 'logpv_add_axy'].iloc[0],
                                  genenames[i], fontsize=gfont)
                elif gstyle == 2:
                    plt.annotate(genenames[i], xy=(d.loc[d[geneid] == i, lfc].iloc[0], d.loc[d[geneid] == i, 'logpv_add_axy'].iloc[0]),
                                 xycoords='data', xytext=(5, -15), textcoords='offset points', size=6,
                                 bbox=dict(boxstyle="round", alpha=0.1),
                                 arrowprops=dict(arrowstyle="wedge,tail_width=0.5", alpha=0.1, relpos=(0, 0)))
                else:
                    print("Error: invalid gstyle choice")
                    sys.exit(1)


def volcano(df="dataframe", lfc=None, pv=None, lfc_thr=(1, 1), pv_thr=(0.05, 0.05), color=("green", "grey", "red"),
            valpha=1, geneid=None, genenames=None, gfont=8, dim=(5, 5), ar=90, dotsize=1, markerdot="o",
            sign_line_v=False, sign_line_h=False, gstyle=1, axtickfontsize=9,
            axtickfontname="Arial", axlabelfontsize=9, axlabelfontname="Arial", axxlabel=None,
            axylabel=None, xlm=None, ylm=None, plotlegend=False, legendpos='best',
            figname='volcano', legendanchor=None,
            legendlabels=['Significant up', 'Not significant', 'Significant down'], theme=None, path='', ret=False, **kwargs):
    _x = r'$ \log_{2}(\mathrm{Fold Change})$'
    _y = r'$ -\log_{10}(\mathrm{p-value})$'
    color = color
    ax = kwargs.get('ax')
    if ax:
        plt.sca(ax)
    # check if dataframe contains any non-numeric character
    assert check_for_nonnumeric(df[lfc]) == 0, 'dataframe contains non-numeric values in lfc column'
    assert check_for_nonnumeric(df[pv]) == 0, 'dataframe contains non-numeric values in pv column'
    # this is important to check if color or logpv exists and drop them as if you run multiple times same command
    # it may update old instance of df
    df = df.drop(['color_add_axy', 'logpv_add_axy'], axis=1, errors='ignore')
    assert len(set(color)) == 3, 'unique color must be size of 3'
    df.loc[(df[lfc] >= lfc_thr[0]) & (df[pv] < pv_thr[0]), 'color_add_axy'] = color[0]  # upregulated
    #df.loc[(df[lfc] >= lfc_thr[0]) & (df[pv] < pv_thr[0]), 'size_add_axy'] = dotsize[0]
    df.loc[(df[lfc] <= -lfc_thr[1]) & (df[pv] < pv_thr[1]), 'color_add_axy'] = color[2]  # downregulated
    #df.loc[(df[lfc] <= -lfc_thr[1]) & (df[pv] < pv_thr[1]), 'size_add_axy'] = dotsize[2]
    df['color_add_axy'].fillna(color[1], inplace=True)  # intermediate
    #df['size_add_axy'].fillna(dotsize[1], inplace=True)  # intermediate
    df['logpv_add_axy'] = -(np.log10(np.array(df[pv].values.astype(float))))
    # plot
    assign_values = {col: i for i, col in enumerate(color)}
    color_result_num = [assign_values[i] for i in df['color_add_axy']]

    #assert len(set(color_result_num)) == 3, \
    #    'either significant or non-significant genes are missing; try to change lfc_thr or pv_thr to include ' \
    #    'both significant and non-significant genes'
    if theme == 'dark':
        plt.style.use('dark_background')
    #plt.subplots(figsize=dim)
    if plotlegend:
        s = plt.scatter(df[lfc], df['logpv_add_axy'], c=color_result_num, cmap=ListedColormap(color), alpha=valpha,
                        s=dotsize, marker=markerdot)
        assert len(legendlabels) == 3, 'legendlabels must be size of 3'
        plt.legend(handles=s.legend_elements()[0], labels=legendlabels, loc=legendpos, bbox_to_anchor=legendanchor)
    else:
        plt.scatter(df[lfc], df['logpv_add_axy'], c=color_result_num, cmap=ListedColormap(color), alpha=valpha,
                    s=dotsize, marker=markerdot)
    if sign_line_h:
        plt.axhline(y=-np.log10(pv_thr[0]), linestyle='--', color='black', linewidth=1)
    if sign_line_v:
        plt.axvline(x=lfc_thr[0], linestyle='--', color='black', linewidth=1)
        plt.axvline(x=-lfc_thr[1], linestyle='--', color='black', linewidth=1)
    gene_plot(df, geneid, lfc, lfc_thr, pv_thr, genenames, gfont, pv, gstyle)

    if axxlabel:
        _x = axxlabel
    if axylabel:
        _y = axylabel

    plt.xlabel(_x, fontsize=axlabelfontsize, fontname=axlabelfontname)
    plt.ylabel(_y, fontsize=axlabelfontsize, fontname=axlabelfontname)
    if xlm:
        plt.xlim(left=xlm[0], right=xlm[1])
        plt.xticks(fontsize=axtickfontsize, rotation=ar, fontname=axtickfontname)

    else:
        plt.xticks(fontsize=axtickfontsize, rotation=ar, fontname=axtickfontname)
    if ylm:
        plt.ylim(bottom=ylm[0], top=ylm[1])
        plt.yticks(np.arange(ylm[0], ylm[1], ylm[2]),  fontsize=axtickfontsize, rotation=ar, fontname=axtickfontname)
    else:
        plt.yticks(fontsize=axtickfontsize, rotation=ar, fontname=axtickfontname)
    
    if ret:
        return plt.gca()
    else:
        plt.savefig(f"{path}/{figname}.png", bbox_inches='tight', dpi=400)
        plt.savefig(f"{path}/{figname}.pdf", bbox_inches='tight', dpi=400)
        plt.clf()
        plt.close()

In [ ]:
path = 'E:/YandexDisk/bbd/fmba/dnam/processed/special_63/GSE220622/gen'
path_figures = 'E:/YandexDisk/pydnameth/draft/13_fmba_cvd_dnam/figures'

df_ages = pd.read_excel(f"{path}/ages_corrected.xlsx", index_col=0)
df_ages['Features'] = df_ages.index
df_ages['Mann-Whitney'] = -np.log10(df_ages['mannwhitneyu_pval_fdr_bh'].values)
df_ages['ANCOVA'] = -np.log10(df_ages['ancova_Status[T.Control]_pval_fdr_bh'].values)

df_metrics = pd.read_excel(f"{path}/metrics.xlsx", index_col=0)
df_metrics['Features'] = df_metrics.index
df_metrics['Mann-Whitney'] = -np.log10(df_metrics['mannwhitneyu_pval_fdr_bh'].values)
df_metrics['ANCOVA'] = -np.log10(df_metrics['ancova_Status[T.Control]_pval_fdr_bh'].values)

df_scores = pd.read_excel(f"{path}/scores.xlsx", index_col=0)
df_scores.index = df_scores.index.astype(str)
df_scores.index = df_scores.index.str.replace(' (EpiScores)', '')
df_scores['Features'] = df_scores.index
df_scores[r"$-\log_{10}(\mathrm{Mann-Whitney\ p-value})$"] = -np.log10(df_scores['mannwhitneyu_pval_fdr_bh'].values)
df_scores[r"$-\log_{10}(\mathrm{ANCOVA\ p-value})$"] = -np.log10(df_scores['ancova_Status[T.Control]_pval_fdr_bh'].values)
episcores_nonsign = df_scores.index[(df_scores['mannwhitneyu_pval_fdr_bh'] > 0.05) & (df_scores['ancova_Status[T.Control]_pval_fdr_bh'] > 0.05)]
episcores_sign = df_scores.index.difference(episcores_nonsign)

df_dmps = pd.read_csv(f"{path}/GSEA(ebayes)_group_orgn_limma.csv", index_col=0)
df_dmps["CpG"] = df_dmps.index.values
df_dmps.sort_values(["adj.P.Val"], ascending=[True], inplace=True)
df_dmps['print'] = df_dmps.apply(lambda row: f"{row['CpG'].split('_')[0]}", axis=1)
df_dmps['log_pval'] = -np.log10(df_dmps["adj.P.Val"])

colors_tests = {
    'Mann-Whitney': 'fuchsia',
    'ANCOVA': 'cyan'
}


fig = plt.figure(
        figsize=(15, 20),
        layout="constrained"
    )

axs = fig.subplot_mosaic(
    [
        ['ages', 'scores'],
        ['metrics', 'scores'],
        ['volcano', 'volcano'],
    ],
    height_ratios=[4.25, 0.75, 5],
    width_ratios=[1, 4],
    gridspec_kw={
        # "bottom": 0.14,
        # "top": 0.95,
        # "left": 0.1,
        # "right": 0.5,
        # "wspace": 0.33,
        # "hspace": 0.01,
    },
)

sns.set_theme(style='whitegrid')

df_fig = df_ages.copy()
df_fig = df_fig.melt(id_vars='Features', value_vars=['Mann-Whitney', 'ANCOVA'], var_name='Test', value_name=r"$-\log_{10}(\mathrm{p-value})$")
barplot = sns.barplot(
    data=df_fig,
    y='Features',
    x=r"$-\log_{10}(\mathrm{p-value})$",
    edgecolor='black',
    palette=colors_tests,
    hue='Test',
    hue_order=['Mann-Whitney', 'ANCOVA'],
    ax=axs['ages'],
)
axs['ages'].set_ylabel('')
axs['ages'].axvline(-np.log10(0.05), color="red", linestyle="dotted", linewidth=2.0)
axs['ages'].text(-9.5, 1, 'A', fontsize=30, fontfamily='arial')

df_fig = df_metrics.copy()
df_fig = df_fig.melt(id_vars='Features', value_vars=['Mann-Whitney', 'ANCOVA'], var_name='Test', value_name=r"$-\log_{10}(\mathrm{p-value})$")
barplot = sns.barplot(
    data=df_fig,
    y='Features',
    x=r"$-\log_{10}(\mathrm{p-value})$",
    edgecolor='black',
    palette=colors_tests,
    hue='Test',
    hue_order=['Mann-Whitney', 'ANCOVA'],
    ax=axs['metrics'],
)
axs['metrics'].set_ylabel('')
axs['metrics'].axvline(-np.log10(0.05), color="red", linestyle="dotted", linewidth=2.0)
axs['ages'].text(-9.5, 38, 'B', fontsize=30, fontfamily='arial')

scatter = sns.scatterplot(
    data=df_scores.loc[episcores_nonsign, :],
    x=r"$-\log_{10}(\mathrm{Mann-Whitney\ p-value})$",
    y=r"$-\log_{10}(\mathrm{ANCOVA\ p-value})$",
    linewidth=0.25,
    alpha=0.75,
    edgecolor="k",
    s=15,
    color='gray',
    ax=axs['scores'],
)
axs['scores'].axvline(-np.log10(0.05), color='lightgray', linestyle="-", linewidth=2.5, zorder=0)
axs['scores'].axvline(-np.log10(0.05), color='fuchsia', linestyle="dotted", linewidth=2.0, zorder=1)
axs['scores'].axhline(-np.log10(0.05), color='lightgray', linestyle="-", linewidth=2.5, zorder=0)
axs['scores'].axhline(-np.log10(0.05), color='cyan', linestyle="dotted", linewidth=2.0, zorder=1)
scatter = sns.scatterplot(
    data=df_scores.loc[episcores_sign, :],
    x=r"$-\log_{10}(\mathrm{Mann-Whitney\ p-value})$",
    y=r"$-\log_{10}(\mathrm{ANCOVA\ p-value})$",
    linewidth=0.25,
    alpha=0.75,
    edgecolor="k",
    s=25,
    color='red',
    ax=axs['scores'],
)
texts = []
for i, row in df_scores.loc[episcores_sign, :].iterrows():
    texts.append(axs['scores'].text(row[r"$-\log_{10}(\mathrm{Mann-Whitney\ p-value})$"], row[r"$-\log_{10}(\mathrm{ANCOVA\ p-value})$"], row['Features'], fontsize=9))
adjust_text(texts, arrowprops=dict(arrowstyle='-', color='black'), ax=axs['scores'])
axs['scores'].set_xlim(0, 11.8)
axs['scores'].set_ylim(0, 11)
axs['scores'].text(-0.9, 10.61, 'C', fontsize=30, fontfamily='arial')

volc = volcano(
    df=df_dmps,
    lfc='logFC',
    pv='adj.P.Val',
    pv_thr=(0.05, 0.05),
    lfc_thr=(0.0, 0.0),
    path=f"{path}",
    geneid='print',
    axtickfontsize=18,
    axlabelfontsize=18,
    gfont=18,
    gstyle=2,
    sign_line_h=True,
    sign_line_v=False,
    ar=0,
    color=('green', 'gray', 'red'),
    dim=(4, 4), 
    ret=True,
    ax=axs['volcano'],
    dotsize=15,
)
axs['volcano'].set_ylim(0, 10.5)
axs['volcano'].text(-0.185, 10.1, 'D', fontsize=30, fontfamily='arial')

fig.savefig(f"{path_figures}/4/fig.png", bbox_inches='tight', dpi=200)
fig.savefig(f"{path_figures}/4/fig.pdf", bbox_inches='tight')
plt.close(fig)